# Prom Sort

This module shows an example of how to use the "Prom Sort" module.

In [1]:
%config Completer.use_jedi = False

## Definition of inputs and problem formalization

In [2]:
import pandas as pd
from modular_parts.M3_PrometheePreference import compute_preference_indices
from modular_parts.M8_PrometheeOutrankingFlows import calculate_promethee_outranking_flows
from modular_parts.M18_PromSort import calculate_promsort_sorted_alternatives
from core.enums import Direction, GeneralCriterion, FlowType


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)
profiles_performances = pd.DataFrame([[12, 80, 16],
                                      [15, 68, 18],
                                      [18, 55, 20]], index=profiles, columns=criteria)


categories = [f"C{i}" for i in range(1, 5)]  # boundary

## Required data

### Required preferences

In [3]:
aggregated_altenatives_vs_profiles_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights,
                                                                    profiles_performances)

aggregated_profiles_vs_profiles_preferences, _ = compute_preference_indices(profiles_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)

In [4]:
aggregated_altenatives_vs_profiles_preferences[0]

,p1,p2,p3
a1,0.8,0.50,0.5
a2,0.0,0.00,0.0
a3,0.5,0.50,0.0
a4,1.0,0.96,0.0
a5,1.0,0.92,0.5
a6,0.5,0.50,0.5
a7,0.7,0.54,0.5


In [5]:
aggregated_altenatives_vs_profiles_preferences[1]

,a1,a2,a3,a4,a5,a6,a7
p1,0.0,1.0,0.00,0.0,0.0,0.5,0.00
p2,0.2,1.0,0.38,0.0,0.0,0.5,0.30
p3,0.5,1.0,0.50,0.0,0.0,0.5,0.38


In [6]:
aggregated_profiles_vs_profiles_preferences

,p1,p2,p3
p1,0.0,0.0,0.0
p2,1.0,0.0,0.0
p3,1.0,1.0,0.0


### Required flows

In [7]:
alternatives_prometheeI_flows = calculate_promethee_outranking_flows(aggregated_altenatives_vs_profiles_preferences,
                                                                     flow_type=FlowType.BASIC)
profiles_prometheeI_flows = calculate_promethee_outranking_flows(aggregated_profiles_vs_profiles_preferences,
                                                                 flow_type=FlowType.BASIC)

In [8]:
alternatives_prometheeI_flows

,positive,negative
a1,0.600000,0.233333
a2,0.000000,1.000000
a3,0.333333,0.293333
a4,0.653333,0.000000
a5,0.806667,0.000000
a6,0.500000,0.500000
a7,0.580000,0.226667


In [9]:
profiles_prometheeI_flows

,positive,negative
p1,0.0,1.0
p2,0.5,0.5
p3,1.0,0.0


## Usage of PromSort

In [10]:
first_step_assignments, final_step_assignments = calculate_promsort_sorted_alternatives(
    categories,
    alternatives_prometheeI_flows,
    profiles_prometheeI_flows,
    preference_thresholds,
    profiles_performances,
    criteria_directions,
    cut_point=0,
    assign_to_better_class=True)

Imprecise assignments:

In [11]:
first_step_assignments

,worse,better
a1,C3,C3
a2,C1,C2
a3,C2,C2
a4,C3,C3
a5,C3,C3
a6,C2,C2
a7,C3,C3


Precise assignments:

In [12]:
final_step_assignments

a1    C3
a2    C1
a3    C2
a4    C3
a5    C3
a6    C2
a7    C3
dtype: object